In [ ]:
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', None)
# pd.set_option('display.expand_frame_repr', False)  # 
import os
candidates = [ 'lopo_jina_result_record.xlsx',
 'lopo_ml_enhance_result_record.xlsx',
 'lopo_rnn_result.xlsx',
 'lopo_LSI.xlsx',
 'lopo_rag_result.xlsx',
 'lopo_codebert_result_record.xlsx'
 ]
# os.listdir()

In [ ]:
import pandas as pd
import sklearn.metrics as metrics
result = {
    'model_name':[],
    'data_name':[],
    # 'enhance_type':[],
    # 'enhance_llm':[],
    'precision':[],
    'recall':[],
    'f1':[],
    'f2':[],
}
for i in range(len(candidates)):
    df = pd.read_excel(candidates[i])
    if 'socre' in df.columns:
        df['score'] = df['socre']
    print(candidates[i])
    if 'socre' in df.columns:
        df['score'] = df['socre']
    
    if 'pred' not in df.columns:
        t = df['score'].mean()
        df['pred'] = df['score'].apply(lambda x: 1 if x >= t else 0)
    for d in ['EBT','iTrust','eTOUR','RETRO']:
        df_d = df[df['data_name'] == d]
        # accuracy precision recall f1 f2
        precision = metrics.precision_score(df_d['label'], df_d['pred'])
        recall = metrics.recall_score(df_d['label'], df_d['pred'])
        f1 = metrics.f1_score(df_d['label'], df_d['pred'])
        f2 = metrics.fbeta_score(df_d['label'], df_d['pred'], beta=2)
        result['model_name'].append(candidates[i].split('_')[1])
        result['data_name'].append(d)
        result['precision'].append(precision)
        result['recall'].append(recall)
        result['f1'].append(f1)
        result['f2'].append(f2)
df_result1 = pd.DataFrame(result)

df_result = df_result1[['model_name','precision','recall','f1','f2']].groupby(by=['model_name']).mean()
df_result

In [ ]:
import sklearn.metrics as metrics

result = {
    'model_name':[],
    'data_name':[],
    # 'enhance_type':[],
    # 'enhance_llm':[],
    'seed':[],
    'precision':[],
    'recall':[],
    'f1':[],
    'f2':[],
}
for i in range(len(candidates)):
    
    df = pd.read_excel(candidates[i])
    
    # 如果存在socre字段
    if 'socre' in df.columns:
        df['score'] = df['socre']
    
    if 'pred' not in df.columns:
        if 'LSI' in candidates[i]:
            t = 0.68
        elif 'ml' in candidates[i]:
            t = 0.48
        else:
            t = df['score'].mean()
        df['pred'] = df['score'].apply(lambda x: 1 if x >= t else 0)
    for d in ['EBT','iTrust','eTOUR','RETRO']:
        for seed in range(2014,2019):
            if 'seed' in df.columns and 'rag' not in candidates[i]:
                df_d = df[(df['data_name'] == d)&(df['seed'] == seed)]
            else:
                df_d = df[(df['data_name'] == d)]

            # accuracy precision recall f1 f2
            precision = metrics.precision_score(df_d['label'], df_d['pred'])
            recall = metrics.recall_score(df_d['label'], df_d['pred'])
            f1 = metrics.f1_score(df_d['label'], df_d['pred'])
            f2 = metrics.fbeta_score(df_d['label'], df_d['pred'], beta=2)
            result['model_name'].append(candidates[i].split('_')[1])
            result['data_name'].append(d)
            result['seed'].append(seed)
            result['precision'].append(precision)
            result['recall'].append(recall)
            result['f1'].append(f1)
            result['f2'].append(f2)
df_result1 = pd.DataFrame(result)
flag = True
df_result = df_result1[['model_name','seed','precision','recall','f1','f2']].groupby(by=['model_name','seed']).mean()
df_result

In [ ]:
from scipy.stats import wilcoxon

def perform_wilcoxon_test(before, after):

    stat, p = wilcoxon(before, after, alternative='less')
    return stat, p
p_data = df_result.reset_index()
p_data

In [ ]:
# The first and second place of each metric
# presision
before = p_data[p_data['model_name']=='jina']['precision'].values
after = p_data[p_data['model_name']=='rag']['precision'].values
stat,p_value = perform_wilcoxon_test(before,after)
print('precision',p_value)
# recall
before = p_data[p_data['model_name']=='codebert']['recall'].values
after = p_data[p_data['model_name']=='jina']['recall'].values
stat,p_value = perform_wilcoxon_test(before,after)
print('recall',p_value)
# f1
before = p_data[p_data['model_name']=='rag']['f1'].values
after = p_data[p_data['model_name']=='jina']['f1'].values
stat,p_value = perform_wilcoxon_test(before,after)

print('f1',p_value)
# f2
before = p_data[p_data['model_name']=='rag']['f2'].values
after = p_data[p_data['model_name']=='jina']['f2'].values
stat,p_value = perform_wilcoxon_test(before,after)

print('f2',p_value)


In [ ]:
# jina  codebert
# presision
before = p_data[p_data['model_name']=='codebert']['precision'].values
after = p_data[p_data['model_name']=='jina']['precision'].values
stat,p_value = perform_wilcoxon_test(before,after)
print('precision',p_value)
# recall
before = p_data[p_data['model_name']=='codebert']['recall'].values
after = p_data[p_data['model_name']=='jina']['recall'].values
stat,p_value = perform_wilcoxon_test(before,after)
print('recall',p_value)
# f1
before = p_data[p_data['model_name']=='codebert']['f1'].values
after = p_data[p_data['model_name']=='jina']['f1'].values
stat,p_value = perform_wilcoxon_test(before,after)
print('f1',p_value)
# f2
before = p_data[p_data['model_name']=='codebert']['f2'].values.tolist()
after = p_data[p_data['model_name']=='jina']['f2'].values.tolist()
stat,p_value = perform_wilcoxon_test(before,after)
print('f2',p_value)
